# Cleaning Data with PySpark

Working with data is tricky - working with millions or even billions of rows is worse. Did you receive some data processing code written on a laptop with fairly pristine data? Chances are you’ve probably been put in charge of moving a basic data process from prototype to production. You may have worked with real world datasets, with missing fields, bizarre formatting, and orders of magnitude more data. Even if this is all new to you, this course helps you learn what’s needed to prepare data processes using Python with Apache Spark. You’ll learn terminology, methods, and some best practices to create a performant, maintainable, and understandable data processing platform.

# DataFrame details

A review of DataFrame fundamentals and the importance of data cleaning.

## Defining a schema
Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

- Name
- Age
- City

The Name and City columns are StringType() and the Age column is an IntegerType().

In [2]:
# Import the pyspark.sql.types library
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city', StringType(), False)
])

## Using lazy processing
Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to Spark not performing any transformations until an action is requested.

For this exercise, we'll be defining a Data Frame (aa_dfw_df) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparent.

In [3]:
import pyspark.sql.functions as F

# File Path
file_path = ".../data/datacamp/flights_csv/AA_DFW_2018.csv"

# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load(file_path).repartition(12)

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       07/31/2018|         1405|                          194|    lax|
|       08/27/2018|         1566|                           60|    okc|
|       02/21/2018|         2181|                          198|    san|
|       05/24/2018|         2386|                           65|    iah|
|       07/04/2018|         0520|                          157|    phx|
|       03/14/2018|         1424|                          176|    san|
|       07/16/2018|         1291|                          183|    iad|
|       05/08/2018|         0423|                          147|    clt|
|       01/06/2018|         1866|                          149|    mco|
|       02/28/2018|         2275|                          167|    orf|
|       06/15/2018|         2686|                          136| 

## Saving a DataFrame in Parquet format
When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The Parquet format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

In this exercise, we're going to practice creating a new Parquet file and then process some data from it.

In [4]:
# File Path
file_path = ".../data/datacamp/flights_csv/"

# Load the CSV file
df1 = spark.read.format('csv').options(Header=True).load(file_path + "AA_DFW_2018.csv").repartition(12)
df2 = spark.read.format('csv').options(Header=True).load(file_path + "AA_DFW_2017.csv").repartition(12)

# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)\
        .withColumnRenamed("Date (MM/DD/YYYY)", "Date")\
        .withColumnRenamed("Flight Number", "FlightNumber")\
        .withColumnRenamed("Destination Airport", "DestinationAirport")\
        .withColumnRenamed("Actual elapsed time (Minutes)", "ActualElapsedTime")

# Save the df3 DataFrame in Parquet format
df3.repartition(12).write.parquet(".../data/datacamp/flights_parquet", mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet(".../data/datacamp/flights_parquet").count())

df1 Count: 119910
df2 Count: 139358
259268


## SQL and Parquet
Parquet files are perfect as a backing data store for SQL queries in Spark. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

In [5]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('.../data/datacamp/flights_parquet')

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(ActualElapsedTime) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 151
